## Import Library

In [1]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from pandas_datareader import data as web
import os
import string
import requests

In [12]:
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

In [116]:
Stock_Price_df = pd.read_csv("Stock_Price.csv")

In [117]:
Stock_Price_df.head()

,Attributes,High,Low,Open,Close,Volume,Adj Close
0,Symbols,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-09-26,2158.5400390625,2145.0400390625,2158.5400390625,2146.10009765625,3216170000,2146.10009765625
3,2016-09-27,2161.1298828125,2141.550048828125,2146.0400390625,2159.929931640625,3437770000,2159.929931640625
4,2016-09-28,2172.39990234375,2151.7900390625,2161.85009765625,2171.3701171875,3891460000,2171.3701171875


In [110]:
news_df = pd.read_csv("news.csv")

In [111]:
news_df.head()

,Unnamed: 0,Date,News_title,News_abstract
0,0,2016-09-26,"['Indian Stocks Emerge From the Crowd ', 'Dona...",[' Indian shares are on a roll in 20...
1,1,2016-09-27,"['Takata Air Bags Recalled in South Korea ', '...",[' South Korea’s transport ministry ...
2,2,2016-09-28,"['Photos of the Day:Asia ', 'Missile System Th...",[' A man visits the Forbidden City i...
3,3,2016-09-29,"['Photos of the Day: Asia ', 'Traders to Scour...",[' Bodybuilders compete over their m...
4,4,2016-09-30,['Vietnam to Launch New Benchmark Stock Index ...,[' Vietnam said it will launch VNX A...


In [112]:
news_df['News_title'][0:10]

0    ['Indian Stocks Emerge From the Crowd ', 'Dona...
1    ['Takata Air Bags Recalled in South Korea ', '...
2    ['Photos of the Day:Asia ', 'Missile System Th...
3    ['Photos of the Day: Asia ', 'Traders to Scour...
4    ['Vietnam to Launch New Benchmark Stock Index ...
5    ['U.S. Auto Sales Slipped in September ', 'Nob...
6    ['Derrick Rose’s Trial Begins as Knicks Start ...
7    ['Oregon Standoff Leader Ammon Bundy Testifies...
8    ['North Korea Activity at Nuclear Sites Raises...
9    ['U.S. Carriers to Allow Second Samsung Phone ...
Name: News_title, dtype: object

In [113]:
news_df['News_abstract'][0:10]

0    ['           Indian shares are on a roll in 20...
1    ['           South Korea’s transport ministry ...
2    ['           A man visits the Forbidden City i...
3    ['           Bodybuilders compete over their m...
4    ['           Vietnam said it will launch VNX A...
5    ['           Light-vehicle sales sputtered in ...
6    ['           Knicks guard Derrick Rose’s civil...
7    ['           The leader of a 41-day standoff a...
8    ['           Satellite images showing vehicles...
9    ['           U.S. phone carriers said Friday t...
Name: News_abstract, dtype: object

In [114]:
news_df.shape

(508, 4)

In [118]:
Stock_Price_df.drop(Stock_Price_df.index[:2], inplace=True)
Stock_Price_df.shape

(508, 7)

### Text Mining
##### Data Pre-processing

In [119]:
def news_filter(text):
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in set(string.punctuation))  # remove punctuation
    text = re.sub(r'\d+', '', text)    # remove digits
    text = " ".join(text.split())     # remove white space
    stop_words = set(stopwords.words('english'))
    tokens1 = word_tokenize(text)
    text1 = [i for i in tokens1 if not i in stop_words] # remove stop words
    text1 = ' '.join(text1) # Back to string
    lemmatizer=WordNetLemmatizer()
    tokens2 = word_tokenize(text1)
    text2 = [lemmatizer.lemmatize(word) for word in tokens2] # lemmatizer
#     text2 = ' '.join(text2) # Back to string
#     stemmer= PorterStemmer()
#     tokens3=word_tokenize(text2)
#     text3 = [stemmer.stem(word) for word in tokens3] # stemmer
    
    return text2

In [120]:
def raw_to_filter(df):
#     cleaned_title = []
    for index, day_news in enumerate(df['News_title']):
        day_news = day_news[1:-1]
        filter_day_news = news_filter(day_news)
        news_df['News_title'][index] = " ".join(filter_day_news)
    for index, day_abs in enumerate(df['News_abstract']):
        day_abs = day_abs[1:-1]
        filter_day_abs = news_filter(day_abs)
        news_df['News_abstract'][index] = " ".join(filter_day_abs)
    return None

In [121]:
raw_to_filter(news_df)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [122]:
news_df.head(10)

,Unnamed: 0,Date,News_title,News_abstract
0,0,2016-09-26,indian stock emerge crowd donald trump attack ...,indian share roll many investor say country be...
1,1,2016-09-27,takata air bag recalled south korea ペレス前イスラエル大...,south korea ’ transport ministry ordered dozen...
2,2,2016-09-28,photo dayasia missile system downed mh said ru...,man visit forbidden city beijing hindu devotee...
3,3,2016-09-29,photo day asia trader scour japan ’ bondbuying...,bodybuilder compete muscle championship nepal ...
4,4,2016-09-30,vietnam launch new benchmark stock index orego...,vietnam said launch vnx allshare oct include s...
5,5,2016-10-03,u auto sale slipped september nobel prize medi...,lightvehicle sale sputtered u last month despi...
6,6,2016-10-04,derrick rose ’ trial begin knicks start presea...,knicks guard derrick rose ’ civil trial regard...
7,7,2016-10-05,oregon standoff leader ammon bundy testifies t...,leader day standoff national wildlife refuge o...
8,8,2016-10-06,north korea activity nuclear site raise specul...,satellite image showing vehicle people around ...
9,9,2016-10-07,u carrier allow second samsung phone replaceme...,u phone carrier said friday allowing customer ...


Write out to .csv for future steps

In [123]:
news_df.to_csv("newscleaned1_kc3214.csv", header = True)

In [124]:
print("Numbers of collected daily news: ", news_df.shape[0])
print("Number of collected daily stock price:", Stock_Price_df.shape[0])

Numbers of collected daily news:  508
Number of collected daily stock price: 508


________________

In [172]:
news_df = pd.read_csv("newscleaned1.csv")

As we can see the first round Text Mining did not delete all useless information<br> 
so we are tying to go further on text mining steps.

In [173]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Count Words frequency, and create a high frequency set

In [208]:
# For News Title
word_vectorizer_tit = CountVectorizer(analyzer='word', stop_words='english', min_df=5)
Smatrix = word_vectorizer_tit.fit_transform(news_df['News_title'])
frequencies = sum(Smatrix).toarray()[0]
freq_names = word_vectorizer_tit.get_feature_names()
print(freq_names[:100])
freq = pd.DataFrame(frequencies, index = freq_names, columns=['frequency'])
freq

['aaron', 'ab', 'abandon', 'abandoned', 'abandoning', 'abb', 'abbas', 'abbott', 'abbvie', 'abc', 'abe', 'abercrombie', 'ability', 'able', 'aboard', 'abode', 'abortion', 'abraaj', 'abroad', 'absence', 'absolute', 'absolutely', 'absorb', 'abu', 'abuse', 'abusive', 'ac', 'aca', 'academic', 'academy', 'accelerate', 'accelerated', 'accelerates', 'accelerating', 'accenture', 'accept', 'acceptance', 'accepts', 'access', 'accessible', 'accessory', 'accident', 'accidental', 'accord', 'according', 'account', 'accountability', 'accountable', 'accountant', 'accounting', 'accusation', 'accuse', 'accused', 'accuser', 'accuses', 'ace', 'achievement', 'ackman', 'acknowledges', 'aclu', 'acosta', 'acquire', 'acquired', 'acquires', 'acquiring', 'acquisition', 'acquitted', 'act', 'acted', 'actelion', 'acting', 'action', 'active', 'activision', 'activism', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adam', 'adapt', 'adapts', 'add', 'added', 'addict', 'addicted', 'addiction', 'a

,frequency
aaron,16
ab,17
abandon,18
abandoned,9
abandoning,5
abb,7
abbas,7
abbott,10
abbvie,7
abc,18


In [207]:
# For News Abstract
word_vectorizer_abs = CountVectorizer(analyzer='word', stop_words='english', min_df=5)
Smatrix_abs = word_vectorizer_abs.fit_transform(news_df['News_abstract'])
frequencies_abs = sum(Smatrix_abs).toarray()[0]
freq_names_abs = word_vectorizer_abs.get_feature_names()
print(freq_names_abs[:100])
freq = pd.DataFrame(frequencies_abs, index = freq_names_abs, columns=['frequency'])
freq

['aaron', 'ab', 'abandon', 'abandoned', 'abandoning', 'abated', 'abating', 'abb', 'abbas', 'abbey', 'abbott', 'abbvie', 'abc', 'abdel', 'abducted', 'abduction', 'abdul', 'abdussalaam', 'abe', 'abercrombie', 'aberdeen', 'abide', 'abigail', 'ability', 'able', 'abnormal', 'aboard', 'abolish', 'aborted', 'abortion', 'abound', 'aboutface', 'abraaj', 'abraham', 'abrams', 'abroad', 'abrupt', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'absorbed', 'abstract', 'absurd', 'abu', 'abundance', 'abundant', 'abuse', 'abused', 'abuser', 'abusing', 'abusive', 'aca', 'academia', 'academic', 'academy', 'accelerate', 'accelerated', 'accelerates', 'accelerating', 'acceleration', 'accelerator', 'accent', 'accenture', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessible', 'accession', 'accessory', 'accident', 'accidental', 'accidentally', 'acclaimed', 'accolade', 'accommodate', 'accommodation', 'accommodative', 'accompanied', 'ac

,frequency
aaron,29
ab,15
abandon,46
abandoned,56
abandoning,37
abated,8
abating,6
abb,6
abbas,12
abbey,5


Delete the words frequency < 2

In [209]:
high_freq_word_news = set(freq_names)
high_freq_word_abs = set(freq_names_abs)

In [212]:
def delet_low_fre(df, high_freq_word_news, high_freq_word_abs):
    for index, day_news in  enumerate(df['News_title']):
        news_df['News_title'][index] = " ".join( set(day_news.split()) - (set(day_news.split()) - high_freq_word_news))
        
    for index, day_abs in  enumerate (df['News_abstract']):
        news_df['News_abstract'][index] = " ".join( set(day_abs.split()) - (set(day_abs.split()) - high_freq_word_abs))
        
    return news_df

        

In [213]:
news_df = delet_low_fre(news_df, high_freq_word_news, high_freq_word_abs)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [218]:
news_df = news_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
news_df.head(10)

,Date,News_title,News_abstract
0,2016-09-26,additional skyhigh peres loss export lineup cr...,bare rescind prevent chemical include accompan...
1,2016-09-27,blunt restoration peres loss wealth list fisch...,version oppose loss export according list food...
2,2016-09-28,better lyft chemical peres loss knew robert li...,assuming chemical theatre loss export conducte...
3,2016-09-29,viral peres loss sound list credit pass rip la...,spurring mishandled export according transpare...
4,2016-09-30,peres loss reviewing list robert stance charlo...,fled version blunt persisted include peres los...
5,2016-10-03,better bone slipped loss empire wealth path so...,bone prevent slipped chemical loss empire acco...
6,2016-10-04,viking loss va list food los obamacare erdogan...,prevent hightech viking loss stretch conducted...
7,2016-10-05,chemical peres reduce reviewing credit currenc...,version mishandled theatre floundering loss ex...
8,2016-10-06,better additional william guest siri direct le...,version guest prevent slipped oppose include l...
9,2016-10-07,better chemical escape empire stance fischer c...,rural flash rushing prop slipped chemical incl...


In [219]:
news_df.to_csv("newscleaned2_kc3214.csv", header = True)